Autor: Miguel Ángel Medina Ramírez
<br>Correo: miguel.medina108@alu.ulpgc.es

## Lectura de datos

In [43]:
import pandas as pd
from sklearn.feature_selection import mutual_info_regression,mutual_info_classif
import numpy as np
import json
FILE = 'data'

Leamos los datos para la métrica de información mutua, tratando la variable clase como una variable numérica.

In [44]:
df_numeric_class = pd.read_csv('%s/winequality-white.csv' %FILE, sep=";")
df_numeric_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


## Métrica CFS

El siguiente método calcula la función de métrica para un subconjunto de variables mediante el formato CFS
<p align="center">
  <img src="data/cfs.png" alt="CFS metrica">
</p>
<p align="center">
  Figura 1: Formula CFS
</p>
<br>


In [45]:
def metric_CFS(columns, df, class_attr  = 'quality'):
    correlation = df.corr()
    amount_attr = len(columns)
    
    avg_ca = np.mean([
        abs(correlation[column][class_attr])
        for column in columns
    ])
    
    if amount_attr == 1: 
        avg_aa = 1
    else:
        avg_aa = np.mean([
            abs(correlation[column][other_column])
            for column in columns
            for other_column in columns
            if column != other_column
        ])
    return (amount_attr*avg_ca)/np.sqrt(amount_attr+amount_attr*(amount_attr-1)*avg_aa)

## Métrica MIFS

Aplica el cálculo usando la métrica MIFS, junto al cálculo de la información mutua

<p align="center">
  <img src="data/mifs.png" alt="MIFS">
</p>
<p align="center">
  Figura 2: Formula MIFS
</p>
<br>

<p align="center">
  <img src="data/mutual_information.png" alt="información mutua">
</p>
<p align="center">
  Figura 3: Cálculo de la información mutua
</p>
<br>


En el caso de la métrica MIFS, transformamos los datos para convertir al atributo clase en uno nominal

In [46]:
def metric_MIFS(columns, df, class_attr  = 'quality', beta = 1.):
    value = abs(mutual_info_classif(df.loc[:, columns], df.loc[:, class_attr])[-1])
    if len(columns) == 1:
        return value
    
    sumatory = np.sum(
        abs(
            mutual_info_regression(df.loc[:, columns[:-1]], df.loc[:, columns[-1]])
        )
    )
    return value - beta*sumatory

In [47]:
df_nominal_class = df_numeric_class.copy()
df_nominal_class['quality'] = df_numeric_class['quality'].astype(str)
df_nominal_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   object 
dtypes: float64(11), object(1)
memory usage: 459.3+ KB


La clase nodo almacenará los estados de las combinaciones posibles  junto a su valor de correlación

In [48]:
class Node:
    def __init__(self, columns: list, value: float):
        self.columns = columns
        self.value   = value

    def __lt__(self, other):
        return self.value < other.value

    def __le__(self, other):
        return self.value <= other.value

    def __gt__(self, other):
        return self.value > other.value

    def __ge__(self, other):
        return self.value >= other.value
    
    def __repr__(self):
        return "%s = %s" % (self.columns, self.value)

El método *get_children* se encarga de a partir de un estado crear los N estados siguientes siguiendo esa rama del árbol de búsqueda, eligiendo las combinaciones faltantes. Se escoge como valor que guía la búsqueda la media de todas las correlaciones.

In [49]:
def get_children(father: Node, columns: list, metric: str, df: pd.DataFrame) -> list:
    if not father:
        return [
            Node([column],  metric([column], df)) for column in columns
        ]

    children = []
    for column in columns:
        if column not in father.columns:
            copyFatherColumns = father.columns.copy()
            copyFatherColumns.append(column)
            children.append(
                Node(copyFatherColumns,  metric(copyFatherColumns, df))
            )

    return children

*select_from_variables* se encarga de la exploración del árbol de estados mediante SFS y la métrica correspondiente.

In [50]:
def select_from_variables(df: pd.DataFrame, metric: classmethod, tree_deep: int = 4) -> pd.DataFrame:
    columns = list(df.columns[:-1])
    if len(columns) <= tree_deep:
        raise RuntimeError("the deep of the search tree is higher than the number of columns")

    selection = None
    for _ in range(tree_deep):
        selection = max(get_children(selection, columns, metric, df))

    return selection
     

Para comparar entre las métricas se varia el número de variables seleccionadas de 2 en 2 hasta 8.

## Busqueda mediante MIFS

In [51]:
reportMIFS = {} 
for deep in range(2,10,2):
    reportMIFS['Tree Deep %s' %deep] = select_from_variables(df_nominal_class, metric_MIFS, tree_deep=deep)

## Busqueda mediante CFS

In [52]:
reportCFS = {} 
for deep in range(2,10,2):
    reportCFS['Tree Deep %s' %deep] = select_from_variables(df_numeric_class, metric_CFS, tree_deep=deep)

# Conclusiones

In [60]:
for key, value in reportMIFS.items():
    print("#############",key,"#############")
    print("Atributos que coinciden --> ", [i for i in value.columns if i in reportCFS[key].columns])
print("Atributos en los que difieren: ")
    print("\t CFS   --> ", [i for i in reportCFS[key].columns if i not in value.columns])
    print("\t MIFS  --> ", [i for i in value.columns if i not in reportCFS[key].columns])
    print("\n")

############# Tree Deep 2 #############
Atributos que coinciden -->  ['volatile acidity']
Atributos en los que difieren: 
	 CFS   -->  ['alcohol']
	 MIFS  -->  ['density']


############# Tree Deep 4 #############
Atributos que coinciden -->  ['density', 'volatile acidity']
Atributos en los que difieren: 
	 CFS   -->  ['alcohol', 'chlorides']
	 MIFS  -->  ['fixed acidity', 'sulphates']


############# Tree Deep 6 #############
Atributos que coinciden -->  ['density', 'volatile acidity', 'fixed acidity', 'sulphates', 'chlorides']
Atributos en los que difieren: 
	 CFS   -->  ['alcohol']
	 MIFS  -->  ['citric acid']


############# Tree Deep 8 #############
Atributos que coinciden -->  ['density', 'volatile acidity', 'fixed acidity', 'sulphates', 'chlorides', 'pH']
Atributos en los que difieren: 
	 CFS   -->  ['alcohol', 'total sulfur dioxide']
	 MIFS  -->  ['citric acid', 'free sulfur dioxide']




En suma, se puede apreciar que para un número pequeño de atributos las dos métricas difieren en algunas columnas; el método bajo la métrica del CFS escoge al **alcohol** y bajo la metrica del MIFS a la **densidad**, pero a medida que vamos aumentado el número de variables nos encontramos con una gran número de atributos que coinciden, dejando a un lado un par de variables que serían las siguientes: **alcohol**, **total** **sulfur dioxide**, **citric acid** y **free sulfur dioxide**.